In [1]:
from SimpleCV import *
from ipywidgets import widgets
from IPython.display import display
import pandas as pd
from sklearn import preprocessing
from sklearn.externals import joblib

# Preparing image

In [2]:
# divide number images into pieces of 8x8 and create matrix. image is 32X32
def createMatrix(image):
    size = 4
    m = np.zeros((8,8))

    for x in range(8):
        for y in range(8):
            crop = image.crop(x*size, y*size, size, size)
            pixels = crop.getNumpy()[:,:,0] / 255
            m[x][y] = pixels.sum()
    return m.T.astype(int)

# check if centroid pixel is black. If black return 1 else 0 
def isCentroidPixelBlack(image, centroid):
    img = image.crop(centroid[0], centroid[1], 1, 1)
    color = img.meanColor()
    return 0 if color[0] > 200 else 1

# get the blob from the image
def getBlob(image):
    image.show()
    blobs = image.findBlobs()
    image.draw(blobs, autocolor=True)
    return blobs[0]

def centroidTopBottom(image, centroidY, height):    
    croppedImageTop = image.crop(0,0,height,centroidY)
    blobTop = getBlob(croppedImageTop)
    centroidTop = blobTop.centroid()
    pixelBlackTop = isCentroidPixelBlack(croppedImageTop,centroidTop)
    
    croppedImageBottom = image.crop(0,centroidY, height,(height-centroidY))   
    blobBottom = getBlob(croppedImageBottom)
    centroidBottom = blobBottom.centroid()
    pixelBlackBottom = isCentroidPixelBlack(croppedImageBottom,centroidBottom)

    return (pixelBlackTop, pixelBlackBottom)

In [3]:
def getPreparedImage():
    size = 32
    imagelist = []
    horizontalSplit=[]
    numbers = []

    # get all files and add them to tuple with the postal codes
    originalImage = Image(str(file_selection.value))
    image = originalImage.binarize().dilate().erode()
    invertedImage = image.invert().embiggen( (128,40), Color.WHITE, (3,3))
    imagelist.append((image, invertedImage))
   
    for image in imagelist:
        for i in range(4):
            sizeHeight = 40
            sizeWidth = 32
            normalImage = image[0].crop(i*size, 0, size, size*(i+1))
            invertedImage = image[1].crop(i*sizeWidth, 0, sizeWidth, sizeHeight*(i+1))
            numbers.append((normalImage, invertedImage))
    
    numbersDict = {
        "blob_area" : [],
        "blob_centroid_x" : [],
        "blob_centroid_y" : [],
        "blob_angle" : [],
        "blob_height": [],
        "blob_width": [],
        "blob_amount_contours": [],
        "number_of_holes":[],
        "centroid_is_empty": [],
        "centroid_top_is_empty":[],
        "centroid_bottom_is_empty":[]
    }

    for x in range(8):
        for y in range(8):
            numbersDict["grid_" + str(x) + "_" + str(y)] = []
    
    for number in numbers:
        image = number[0]
        invertedImage = number[1]
        invertedBlobCount = len(invertedImage.findBlobs()) -1
        blob = getBlob(image)
        matrix = createMatrix(image)
        centroid = blob.centroid()
        split = centroidTopBottom(image, centroid[1], blob.height())
        for x in range(8):
            for y in range(8):
                numbersDict["grid_" + str(x) + "_" + str(y)].append(matrix[y][x])
        numbersDict["blob_area"].append(blob.area())
        numbersDict["blob_centroid_x"].append(centroid[0])
        numbersDict["blob_centroid_y"].append(centroid[1])
        numbersDict["blob_angle"].append(blob.angle())
        numbersDict["blob_height"].append(blob.height())
        numbersDict["blob_width"].append(blob.width())
        numbersDict["blob_amount_contours"].append(len(blob.contour()))
        numbersDict["number_of_holes"].append(invertedBlobCount)
        numbersDict["centroid_is_empty"].append(isCentroidPixelBlack(image, centroid))
        numbersDict["centroid_top_is_empty"].append(split[0])
        numbersDict["centroid_bottom_is_empty"].append(split[1])   
        
    numbers = []    
    
    df = pd.DataFrame(numbersDict)

    return df

# Drop columns

In [4]:
scaler = joblib.load('scaler.joblib') 
clf = joblib.load('svcModel.joblib')

def normalize(df):
    df = df.drop(['grid_0_0', 'grid_0_1', 'grid_0_2', 'grid_0_3', 'grid_0_4', 'grid_0_5', 'grid_0_6', 'grid_0_7', 
              'grid_1_0', 'grid_1_1', 'grid_1_2', 'grid_1_3', 'grid_1_4', 'grid_1_5', 'grid_1_6', 'grid_1_7', 
              'grid_6_0', 'grid_6_1', 'grid_6_2', 'grid_6_3', 'grid_6_4', 'grid_6_5', 'grid_6_6', 'grid_6_7',
              'grid_7_0', 'grid_7_1', 'grid_7_2', 'grid_7_3', 'grid_7_4', 'grid_7_5', 'grid_7_6', 'grid_7_7',
              'blob_height', 'blob_amount_contours'], axis=1)        

#     columnsToNormalize =['blob_area', 'blob_centroid_x', 'blob_centroid_y', 'blob_angle','blob_width','number_of_holes',
#               'centroid_is_empty', 'centroid_top_is_empty', 'centroid_bottom_is_empty',
#               'grid_2_0', 'grid_2_1', 'grid_2_2', 'grid_2_3', 'grid_2_4',
#               'grid_2_5', 'grid_2_6', 'grid_2_7', 'grid_3_0', 'grid_3_1',
#               'grid_3_2', 'grid_3_3', 'grid_3_4', 'grid_3_5', 'grid_3_6',
#               'grid_3_7', 'grid_4_0', 'grid_4_1', 'grid_4_2', 'grid_4_3',
#               'grid_4_4', 'grid_4_5', 'grid_4_6', 'grid_4_7', 'grid_5_0',
#               'grid_5_1', 'grid_5_2', 'grid_5_3', 'grid_5_4', 'grid_5_5',
#               'grid_5_6', 'grid_5_7']
    
    df_normalized = df.copy()
#     scaled = scaler.transform(df_normalized[columnsToNormalize])
#     df_normalized[columnsToNormalize] = pd.DataFrame(scaled, columns= columnsToNormalize)
    scaled = scaler.transform(df_normalized)
    df_normalized = pd.DataFrame(scaled, columns=df_normalized.columns)
    return df_normalized

def getPrediction(df):
    df_normalized = normalize(df)
    
    return clf.predict(df_normalized)

# Selecting an image and predict

In [5]:
# Display for showing images
disp = Display(displaytype='notebook')

# create output
output = widgets.Output()

# Find all png files in a folder
files = glob.glob(os.path.join("../dataset-images/", "*.png"))

# Select path
file_input_path = widgets.Text(value="../dataset-images", placeholder="input path", description="Folder")
select_button = widgets.Button(description="Select folder")


# Create a dropdown
file_selection = widgets.Dropdown(
    options=files,
    description='File:'
)

# Create a button
button = widgets.Button(description="Select Image")

# Add widgets to notebook
display(file_input_path)
display(select_button)
display(file_selection)
display(button)
display(output)

# Actions for components
def on_button_clicked(b):
    output.clear_output()
    file = file_selection.value
    if file:
        img = Image(str(file))
        img = img.binarize()
        #prepare the image to be put in the model, returns dataframe
        df = getPreparedImage()
        #gets the predictions for that specific dataframe
        with output:
            img.save(disp)
            print getPrediction(df)
    else:
        with output:
            print 'Please select an image'
    
# Select folder
def on_select_clicked(b):
    files = glob.glob(os.path.join(file_input_path.value, "*.png"))
    file_selection.options = files
    
    
# Bind button click to action
button.on_click(on_button_clicked)
select_button.on_click(on_select_clicked)

Text(value=u'../dataset-images', description=u'Folder', placeholder=u'input path')

Button(description=u'Select folder', style=ButtonStyle())

RHJvcGRvd24oZGVzY3JpcHRpb249dSdGaWxlOicsIG9wdGlvbnM9KCcuLi9kYXRhc2V0LWltYWdlcy83NjIwLnBuZycsICcuLi9kYXRhc2V0LWltYWdlcy83NjI3LnBuZycsICcuLi9kYXRhc2XigKY=


Button(description=u'Select Image', style=ButtonStyle())

Output()

/usr/local/lib/python2.7/dist-packages/SimpleCV/ImageClass.py:1970: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._numpy != "":
